### First part recap ###

Imitation Learning<br><br>

**Meta-learning** for action-based models: <br>
improve classifier by generating training from demonstrations 

### Rollin ###

For each training instance:
- construct an action sequence,
- incrementally transform initial to terminal state.


- Available actions wrt transition system.
<center>
<img src="images/IL_rollin.png">
</center>

### Rollin ###

For each training instance:
- construct an action sequence,
- incrementally transform initial to terminal state.


- Available actions wrt transition system.
<center>
<img src="images/IL_rollin_zoom.png">
</center>

### Rollin ###

Can rollin with <font color="blue">classifier</font>, <font color="green">expert policy</font>, or a mix.

<center>
<img src="images/IL_rollin_classifier.png">
<img src="images/IL_rollin_expert.png">
<img src="images/IL_rollin_mix.png">
</center>

### Explore ###

At each timestep:
- explore alternative actions w.r.t transition system.
<br><br>
<center>
<img src="images/IL_explore.png">
</center>

### Explore ###

Create a new training instance (feature vector):
- cost alternative actions,
- e.g. DAgger
<br><br>
<center>
<img src="images/IL_explore_loss.png">
</center>

### Rollout ###

Or perform rollouts to cost the actions:
- can use non-decomposable loss functions,
- e.g. V-DAgger, SEARN, LOLS
<br><br>
<center>
<img src="images/IL_rollout.png">
</center>

### Rollin ###

Can rollout with <font color="blue">classifier</font>, <font color="green">expert policy</font>, or a mix.
<br><br>
<center>
<img src="images/IL_rollout_classifier.png">
<img src="images/IL_rollout_expert.png">
<img src="images/IL_rollout_mix.png">
</center>

### Part 2: NLP Applications and practical advice

- Applications:
    - dependency parsing 
    - natural language generation
    - semantic parsing

- Practical advice
    - making things faster
    - debugging

<center>
<h2>Applying Imitation Learning to Dependency Parsing</h2>
<h3>[Goldberg and Nivre 2012](http://www.aclweb.org/anthology/C12-1059)</h3>
</center>

### Dependency parsing ###

<img src="images/toBeAnimated/depParse1.png">

To represent the syntax of a sentence as directed labeled arcs between words.
- Labels represent dependencies between words.

### DAgger Reminder ###

<p style="border:3px; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em; font-size: 75%">
\begin{align}
& \textbf{Input:} \; D_{train} = \{(\mathbf{x}^1,\mathbf{y}^1)...(\mathbf{x}^M,\mathbf{y}^M)\}, \; expert\; \pi^{\star}, \; loss \; function \; L,\\
& classifier \; H,\; training\; examples\; \cal E = \emptyset, \; expert\; probability\; \beta=1\\
& \mathbf{while}\; \text{termination condition not reached}\; \mathbf{do}\\
& \quad \color{red}{\text{set} \; rollin \; policy \; \pi^{in} = (1-\beta) \; H + \beta \pi^{\star}}\\
& \quad \mathbf{for} \; (\mathbf{x},\mathbf{y}) \in D_{train} \; \mathbf{do}\\
& \quad \quad \color{red}{\text{rollin to predict} \; \hat \alpha_1\dots\hat \alpha_T  = \pi^{in}(\mathbf{x},\mathbf{y})}\\
& \quad \quad \mathbf{for} \; \hat \alpha_t \in \hat \alpha_1\dots\hat \alpha_T \; \mathbf{do}\\
& \quad \quad \quad \color{red}{\text{ask expert for } \underline{\text{a set of best actions}}\; \{\alpha_{1}^{\star}\dots\alpha_{k}^{\star}\} = \pi^{\star}(\mathbf{x},S_{t-1})} \\
& \quad \quad \quad \text{extract } features=\phi(\mathbf{x},S_{t-1}) \\
& \quad \quad \quad \cal E = \cal E \cup (features,\alpha^{\star})\\
& \quad \text{learn classifier} \; \text{from}\; \cal E\\
& \quad \text{decrease} \; \beta\\
\end{align}
</p>

To apply Imitation Learning on any task, we need to define:
- Transition
- Loss function
- Expert policy

### Transition system? ###

We can assume any transition-based system (e.g. Arc-Eager).

<span class="fragment" data-fragment-index="1"> <b>State:</b> arcs, stack, and buffer.</span>
<br>
<br>
<span class="fragment" data-fragment-index="2"> <b>Action space:</b> <br> <span style="font-variant: small-caps;">Shift, Reduce, Arc-Left</span>, and <span style="font-variant: small-caps;">Arc-Right.</span></span>
<ul>
<li class="fragment" data-fragment-index="3"><span style="font-variant: small-caps;">Arc-Left</span> / <span style="font-variant: small-caps;">Arc-Right</span> combine with arc labels, </li>
<li class="fragment" data-fragment-index="4"> but limited #labels in dependency parsing (~50).</li>
</ul>

The length of the transition sequence is variable.
<br>
<center>
<img src="images/stateTransitExpert.png">
</center><br>
<span style="font-variant: small-caps;">Shift -> Shift -> Arc-Right -> Shift -> ... -> Arc-Left</span>
<br>
<ul>
<li class="fragment" data-fragment-index="2">Bounded (but not fixed) by length of sentence. </li>
<li class="fragment" data-fragment-index="3">In what task would it be fixed? <span class="fragment" data-fragment-index="4"><b>POS tagging!</b></li>
</ul>

<center><h3><span style="font-variant: small-caps;color: white">...</span></h3><br><br>
<img src="images/toBeAnimated/transitionEx_1.png">
</center>
<font size="5">
<b>Stack:</b> -
<br>
<b>Buffer:</b> ROOT, 'economic', 'news', 'had', 'little', 'effect', 'on', 'financial', 'markets', '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Shift</span></h3><br><br>
<img src="images/toBeAnimated/transitionEx_1.png">
</center>
<font size="5">
<b>Stack:</b> <font color="green"><b><i>ROOT</i></b></font>
<br>
<b>Buffer:</b> 'economic', 'news', 'had', 'little', 'effect', 'on', 'financial', 'markets', '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Shift</span></h3><br><br>
<img src="images/toBeAnimated/transitionEx_1.png">
</center>
<font size="5">
<b>Stack:</b> ROOT, <font color="green"><b><i>'economic'</i></b></font>
<br>
<b>Buffer:</b> 'news', 'had', 'little', 'effect', 'on', 'financial', 'markets', '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Arc-Left:</span> amod</h3><br><br>
<img src="images/toBeAnimated/transitionEx_2.png">
</center>
<font size="5">
<b>Stack:</b> ROOT
<br>
<b>Buffer:</b> 'news', 'had', 'little', 'effect', 'on', 'financial', 'markets', '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Shift</span></h3><br><br>
<img src="images/toBeAnimated/transitionEx_2.png">
</center>
<font size="5">
<b>Stack:</b> ROOT, <font color="green"><b><i>'news'</i></b></font>
<br>
<b>Buffer:</b> 'had', 'little', 'effect', 'on', 'financial', 'markets', '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Arc-Left:</span> nsubj</h3><br><br>
<img src="images/toBeAnimated/transitionEx_3.png">
</center>
<font size="5">
<b>Stack:</b> ROOT
<br>
<b>Buffer:</b> 'had', 'little', 'effect', 'on', 'financial', 'markets', '.'
</font>	

<center><h3>and so on</h3><br><br>
<img src="images/toBeAnimated/depParse1.png">
</center>
<font size="5">
<b>Stack:</b> ROOT
<br>
<b>Buffer:</b> -
</font>	

### Loss function? ###

<b>Hamming loss:</b> given predicted arcs, how many parents and labels were incorrectly predicted?
<br>
<ul>
<li class="fragment" data-fragment-index="2"> Directly corresponds to attachment score metrics used to evaluate dependency parsers. </li>
<li class="fragment" data-fragment-index="3"> Costs dependencies, not actions directly.</li>
<li class="fragment" data-fragment-index="4"> Decomposable? <span class="fragment" data-fragment-index="5"><b>Not with this transition model!</b> Cannot score <span style="font-variant: small-caps;">Shift</span> independent of <span style="font-variant: small-caps;">Arc-Right</span>!</span></li>
</ul>

### Expert policy? ###

<center>
<img src="images/oracle-delphi.jpg">
</center>


Returns the best action at the current state by looking at the gold standard assuming future actions are also optimal:

$$\alpha^{\star}=\pi^{\star}(S_t, \mathbf{y}) = \mathop{\arg \min}_{\alpha \in {\cal A}} L(S_t(\alpha,\pi^{\star}),\mathbf{y})$$

### How do we make an expert policy? ###

We can derive a <b>static</b> transition sequence from initial to terminal state using the golden standard.<ul><li>Static expert policy.</li></ul>
<br>
<center>
<span class="fragment" data-fragment-index="1" style="font-variant: small-caps;">Shift -> Shift -> Arc-Right -> Shift -> ... -> Arc-Left</span>
<br>
</center>

Static expert assumes previous actions (rollin) are optimal,<ul><li>or that optimal action independent of rollin actions.</li></ul>

<center>
<img src="images/stateTransit.png" width="90%">
</center>

<center>
<img src="images/stateTransitErrorPart.png" width="90%">
</center>

Static expert has not encountered this state before.<ul><li class="fragment" data-fragment-index="10">Cannot know which is the optimal action. </li><li class="fragment" data-fragment-index="11"> May default to an action (e.g. <span style="font-variant: small-caps;">Shift</span>).</li><ul>

A static expert may be sufficient for tasks where we do not care whether the previous actions were optimal.
<ul><li class="fragment" data-fragment-index="1"> What would such a task be? <span class="fragment" data-fragment-index="2"> <b>POS tagging.</b></span></li></ul>

<table style="border-style: hidden; border-collapse: collapse; padding: 50px"><tr><th>I</th> 
<th>studied</th>
<th>in</th>
<th>London</th>
<th>with</th>
<th>Sebastian</th>
<th>Riedel</th></tr><tr>
<td>PRP</td>
<td>VBD</td>
<td>DET</td>
<td>NNP</td>
<td><font color="red"><b>VBD</b></font></td>
<td>NNP</td>
<td>NNP</td></tr></table>
<br>
- If the previous word is tagged incorrectly, the expert policy's suggestion remains the <b>same</b> and <b>optimal</b>!

### Rollin mistakes in dependency parsing ###

Let's assume that we rollin using the classifier.

<center><h3><span style="font-variant: small-caps; color: green;">Arc-Right:</span> iobj ?</h3><br><br>
<img src="images/toBeAnimated/depParse_mistake_1.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote'
<br>
<b>Buffer:</b> 'her', 'a', 'letter', '.'
</font>	

<center><h3><span style="font-variant: small-caps; color: red;">Shift</span></h3><br><br>
<img src="images/toBeAnimated/depParse_mistake_2.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote', <font color="green"><b><i>'her'</i></b></font>
<br>
<b>Buffer:</b> 'a', 'letter', '.'
</font>	

<center><h3>Default: <span style="font-variant: small-caps;">Shift</span></h3><br><br>
<img src="images/toBeAnimated/depParse_mistake_2.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote', 'her', <font color="green"><b><i>'a'</i></b></font>
<br>
<b>Buffer:</b> 'letter', '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Arc-Left: </span>det</h3><br><br>
<img src="images/toBeAnimated/depParse3.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote', 'her'
<br>
<b>Buffer:</b> 'letter', '.'
</font>	

<center><h3>Default: <span style="font-variant: small-caps;">Shift</span></h3><br><br>
<img src="images/toBeAnimated/depParse3.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote', 'her', <font color="green"><b><i>'letter'</i></b></font>
<br>
<b>Buffer:</b> '.'
</font>	

<br><br><center><h3>Default: <span style="font-variant: small-caps;">Shift</span></h3><br><br>
<img src="images/toBeAnimated/depParse3.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote', 'her', 'letter', <font color="green"><b><i>'.'</i></b></font>
<br>
<b>Buffer:</b> -
</font>	

Static expert policy cannot recover from errors in the rollin.

### Also, what if there are multiple correct transitions? ###

<img src="images/toBeAnimated/depParse_2.png">
<font size="5">
<b>Stack:</b> 'her'
<br>
<b>Buffer:</b> 'a', 'letter', '.'
</font>	

Two possible actions: <span style="font-variant: small-caps;">Reduce</span> 'her' / <span style="font-variant: small-caps;">Shift</span> 'a'

<span class="fragment" data-fragment-index="1">Static expert policy arbitarilly choses. </span>
<ul>
<li class="fragment" data-fragment-index="2"> But chosing any one action indirectly labels the alternative actions as incorrect!</li>
<li class="fragment" data-fragment-index="3"> Leads to noise in the training signal.</li>
</ul>

### Dynamic expert policy ###
Determines best action, by considering the previous actions.<ul><li class="fragment" data-fragment-index="1">Can recover from errors.</li><li class="fragment" data-fragment-index="2">Allows for multiple optimal actions at each time-step.</li><ul>

### Reachable terminal state ###

Reachable terminal state:
- Can be reached through a sequence of expert actions $\alpha_1^{\star}\dots \alpha_T^{\star}$, and
- no further actions can be taken at that state.

- For an optimal reachable terminal state, $L(S_{final}, \mathbf{y}) = 0$.

### How does a dynamic expert policy work? ###

For each possible action at a time-step:
<ul><li style="font-variant: small-caps; color: white;">...</li></ul>
<br>
<center><h3><span style="font-variant: small-caps; color: white;">...</span></h3>
<img src="images/toBeAnimated/depParse3.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote', 'her'
<br>
<b>Buffer:</b> 'letter', '.'
</font>	

### How does a dynamic expert policy work? ###

For each possible action at a time-step:
<ul><li style="font-variant: small-caps;">Arc-Left? Arc-Right? Reduce? Shift?</li></ul>
<br>
<center>
<img src="images/expertPolicyRollOut1.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote', 'her'
<br>
<b>Buffer:</b> 'letter', '.'
</font>	

### How does a dynamic expert policy work? ###

For each possible action at a time-step:
<ul><li style="font-variant: small-caps;">Arc-Left? Arc-Right? Reduce? Shift?</li>
<br>
<li>Determine the reachable terminal state.</li>
<li class="fragment" data-fragment-index="2">Use full rollout with expert?</li></ul>
<br><br><br><br><br>

<center><h3><span style="font-variant: small-caps;">Arc-Left: </span>det?</h3><br><br>
<img src="images/expertPolicyRoll2.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote',
<br>
<b>Buffer:</b> 'letter', '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Arc-Right: </span>dobj</h3><br><br>
<img src="images/expertPolicyRoll3.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote',
<br>
<b>Buffer:</b> '.'
</font>	

<center><h3><span style="font-variant: small-caps;">Arc-Right: </span>p</h3><br><br>
<img src="images/expertPolicyRoll4.png">
</center>
<font size="5">
<b>Stack:</b> 'wrote',
<br>
<b>Buffer:</b> -
</font>	

<img src="images/determReachStates.png">

<center>
<img src="images/determReachStatesScored.png">
</center>

<center>
<img src="images/determReachStatesScoredBest.png">
</center>

### Full rollouts may be expensive ###

Instead of full expert rollout, we may use heuristics!
- i.e. can we estimate how an action will effect the reachable terminal state?

<center><h3><span style="font-variant: small-caps;">Arc-Right:</span> det?</h3>
<img src="images/expertPolicyRollOut_5.png" width="70%">
</center>
<font size="5">
<b>Stack:</b> 'wrote', 'her'
<br>
<b>Buffer:</b> '.'
</font>

<center>
<img src="images/dep_goldMissed.png" width="70%">
</center>

### Results<h5>[Goldberg and Nivre 2012](http://www.aclweb.org/anthology/C12-1059)</h5>
<img src="images/dependResultBars.png">